# DataTales Dashboard Project

This notebook hosts the **DataTales** Dash application. It visualizes the correlations between student lifestyle habits (sleep, social media, diet) and their academic performance.

**Structure:**

- Imports & Setup: initializing the app and styling.

- Data: Loading or generating sample data.

- Components: Reusable UI elements (cards, navbars).

- Pages: Layouts for different sections of the multi-page app.

## SECTION 1: IMPORTS

In [1]:
from dash import Dash, html, dcc, Input, Output, State, callback_context, no_update
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
import webbrowser
import os 

## SECTION 2: APP SETUP & THEME

In [2]:
app = Dash(
    __name__,
    suppress_callback_exceptions=True,
    external_stylesheets=[dbc.themes.BOOTSTRAP, dbc.icons.FONT_AWESOME]
)
app.title = "DataTales"

COLORS = {
    'primary': '#2E86AB', 'secondary': '#1A936F', 'tertiary': '#A23B72',
    'success': '#1A936F', 'warning': '#C44536', 'info': '#118AB2',
    'light': '#F5FBF4', 'light_blue': '#EAF6FA', 'white': '#FFFFFF',
    'dark': '#212529',
    'accent1': '#1982C4', 'accent2': '#118AB2', 'accent3': '#06D6A0',
    'accent4': '#8AC926', 'accent5': '#6A4C93', 'accent6': '#FF9E00',
}

## SECTION 3: DATA LOADING

In [3]:
try:
    df = pd.read_csv("student_habits_performance.csv")
    print("Loaded real data.")
except:
    print("Loaded demo data.")
    np.random.seed(42)
    df = pd.DataFrame({"student_id": range(1, 101), "age": np.random.randint(18, 25, 100), "gender": np.random.choice(["Male", "Female"], 100), "study_hours_per_day": np.random.uniform(1, 8, 100).round(1), "social_media_hours": np.random.uniform(0, 5, 100).round(1), "netflix_hours": np.random.uniform(0, 4, 100).round(1), "part_time_job": np.random.choice(["Yes", "No"], 100), "attendance_percentage": np.random.uniform(60, 100, 100).round(1), "sleep_hours": np.random.uniform(4, 9, 100).round(1), "diet_quality": np.random.choice(["Poor", "Fair", "Good"], 100), "exercise_frequency": np.random.randint(0, 6, 100), "parental_education_level": np.random.choice(["Primary", "Secondary", "Tertiary"], 100), "internet_quality": np.random.choice(["Poor", "Average", "Good"], 100), "mental_health_rating": np.random.randint(1, 11, 100), "extracurricular_participation": np.random.choice(["Yes", "No"], 100), "exam_score": np.random.uniform(40, 100, 100).round(1)})


Loaded real data.


## SECTION 4: REUSABLE HELPER FUNCTIONS

In [4]:
def create_navbar():
    return dbc.NavbarSimple(
        children=[
            dcc.Link([html.I(className="fas fa-home me-2"), "Home"], id="nav-home", href="/", className="nav-link"),
            dcc.Link([html.I(className="fas fa-book-open me-2"), "Introduction"], id="nav-intro", href="/introduction", className="nav-link ms-3"),
            dcc.Link([html.I(className="fas fa-chart-pie me-2"), "Dashboard"], id="nav-dash", href="/dashboard", className="nav-link ms-3"),
            dcc.Link([html.I(className="fas fa-users me-2"), "About Us"], id="nav-about", href="/about-us", className="nav-link ms-3"),
            dcc.Link([html.I(className="fas fa-graduation-cap me-2"), "Acknowledgments"], id="nav-ack", href="/acknowledgments", className="nav-link ms-3"),
        ],
        brand=html.Img(src=app.get_asset_url('logo2.png'), style={'height': '60px'}), brand_href="/", color="white", className="app-navbar px-4", fluid=True
    )

def create_audience_card(title, icon, color, audience_list):
    return dbc.Card(dbc.CardBody([html.I(className=f"fas {icon} fa-2x mb-3", style={'color': color}), html.H6(title, className="fw-bold"), html.Ul(audience_list, className="small text-muted", style={'paddingLeft': '20px', 'textAlign': 'left'})]), className="h-100 text-center", style={'backgroundColor': '#F8F9FA', 'border': 'none', 'borderRadius': '8px'})

def create_variable_flip_card(variable_name, variable_range, description, icon, color):
    return dbc.Col(html.Div([html.Div([html.Div([html.I(className=f"fas {icon} fa-3x mb-3", style={'color': color}), html.H6(variable_name, className="fw-bold mb-2", style={'color': COLORS['dark']}), dbc.Badge(variable_range, pill=True, style={'backgroundColor': color, 'color': '#FFFFFF', 'fontWeight': '600', 'padding': '8px 12px'})
    # --------------------------------
    ], className="flip-card-front"), html.Div([html.H6(variable_name, className="mb-2"), html.P(description)], className="flip-card-back")], className="flip-card-inner")], className="flip-card"), lg=4, md=6, sm=12)
def create_premium_graph_card(title, subtitle, graph_id, icon, color, height='500px'):
    return dbc.Card([dbc.CardBody([html.Div([html.Div(html.I(className=f"fas {icon}", style={'color': 'white'}), style={'backgroundColor': color, 'width': '40px', 'height': '40px', 'borderRadius': '12px', 'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center', 'marginRight': '15px', 'boxShadow': f'0 5px 15px -5px {color}'}), html.Div([html.H5(title, className="fw-bold mb-0", style={'color': COLORS['dark']}), html.Small(subtitle, className="text-muted")])], className="d-flex align-items-center mb-4"), dcc.Graph(id=graph_id, style={'height': height}, config={'displayModeBar': False})])], className="styled-card mb-4")

def create_gradient_stat_card(value, label, icon, gradient_class):
    return dbc.Card(dbc.CardBody([html.I(className=f"fas {icon}", style={'position': 'absolute', 'right': '25px', 'bottom': '15px', 'fontSize': '6rem', 'opacity': '0.15', 'color': '#FFFFFF', 'transform': 'rotate(-15deg)'}), html.Div([html.P(label, className="text-white mb-0", style={'fontWeight': '600', 'textTransform': 'uppercase', 'letterSpacing': '1.5px', 'fontSize': '0.9rem', 'opacity': '0.9'}), html.H2(value, className="text-white mt-2", style={'fontWeight': '900', 'fontSize': '3.2rem'})], style={'position': 'relative', 'zIndex': '2'})]), className=f"mb-4 styled-card {gradient_class}", style={'border': 'none', 'borderRadius': '20px', 'overflow': 'hidden', 'minHeight': '140px'})

## SECTION 5: PAGE LAYOUTS

### 1. Landing page

In [5]:
def create_landing_page():
    return dbc.Container(
        dbc.Row([
            # --- Left Column: Text Content ---
            dbc.Col([
                html.Div([
                    # Subheader styled as a modern tag
                    html.H6("Student Performance Analysis", className="fw-bold mb-3", 
                            style={'color': COLORS['secondary'], 'letterSpacing': '2px', 'textTransform': 'uppercase'}),
                    # Main Header - Bigger and bolder
                    html.H1("Unlock Insights With DataTales", className="display-3 fw-black mb-4", 
                            style={'color': COLORS['dark'], 'fontWeight': '900', 'lineHeight': '1.15'}),
                    # Lead text with better readability
                    html.P("Explore the hidden connections between lifestyle habits and academic performance.",
                           className="lead text-muted mb-5", 
                           style={'fontSize': '1.35rem', 'maxWidth': '550px', 'lineHeight': '1.6'}),
                    
                    # Enhanced Call-to-Action Button
                    dcc.Link(dbc.Button([html.I(className="fas fa-rocket me-2"), "Launch Introduction"], 
                               color="primary", size="lg", 
                               className="px-5 py-3 fw-bold rounded-pill",
                               style={'fontSize': '1.2rem', 'boxShadow': f'0 15px 30px -10px rgba(46, 134, 171, 0.5)', 'transition': 'transform 0.2s'}), 
                             href="/introduction")
                ], className="ps-lg-5 mx-auto mx-lg-0", style={'maxWidth': '700px'}) 
            ], lg=6, className="d-flex flex-column justify-content-center text-center text-lg-start py-5"),
            
            # --- Right Column: Logo with Glow Effect ---
            dbc.Col([
                html.Div([
                    # Subtle background glow element behind the logo
                    html.Div(style={
                        'position': 'absolute', 
                        'width': '70%', 
                        'height': '70%',
                        'background': f'radial-gradient(circle, {COLORS["light_blue"]} 0%, rgba(255,255,255,0) 70%)',
                        'top': '50%', 'left': '50%', 
                        'transform': 'translate(-50%, -50%)', 
                        'zIndex': 0,
                        'opacity': 0.6
                    }),
                    # The Logo with a soft drop shadow
                    html.Img(src=app.get_asset_url('logo.png'), 
                             style={'maxHeight': '650px', 'width': '100%', 'maxWidth': '100%', 
                                    'position': 'relative', 'zIndex': 1,
                                    'filter': 'drop-shadow(0 20px 40px rgba(26, 147, 111, 0.15))'})
                ], style={'position': 'relative', 'display': 'flex', 'justifyContent': 'center', 'alignItems': 'center', 'height': '100%'})
            ], lg=6, className="d-none d-lg-flex align-items-center justify-content-center py-5")
        ], style={'minHeight': 'calc(100vh - 80px)'}),
        fluid=True, 
        style={'background': 'linear-gradient(135deg, #FFFFFF 0%, #F5FBF4 100%)'}
    )

### 2. Introduction

In [6]:
def create_project_introduction():
    hero_section = html.Div([
        dbc.Container([
            dbc.Row([
                dbc.Col([
                    html.H1("Unlocking Student Potential", className="display-4 fw-bolder mb-3", style={'color': COLORS['dark']}),
                    html.P("A data-driven journey into the habits, lifestyle choices, and socioeconomic factors that shape academic success.", 
                           className="lead text-muted mb-4", style={'margin': '0 auto'}),
                    html.Div([
                         dbc.Button([html.I(className="fas fa-chevron-down me-2"), "Explore the Case"], href="#intro-case", external_link=True, color="light", className="fw-bold rounded-pill px-4 py-2", style={'color': COLORS['primary']})
                    ])
                ], lg=10, className="mx-auto")
            ])
        ], fluid=True)
    ], className="intro-hero")

    # --- 1. Introduction to the Case Content ---
    intro_case_content = [
        dbc.Row(
            [
                # Left Column (Main Text)
                dbc.Col(
                    [
                        html.Div([
                             html.H4("The Challenge", className="fw-bold mb-3", style={'color': COLORS['primary']}),
                             html.P(
                                "In the current emerging academic practices, a student's everyday habits, lifestyle choices, and personal experiences can significantly impact their exam scores. Since prior studies have separated variables such as study hours, there is a knowledge gap in understanding the joint effect of these diverse influences (Lakhani, 2024; Mary, 2024).",
                                style={'lineHeight': '1.8', 'fontSize': '1.1rem', 'color': '#444'}
                            ),
                            html.P(
                                "For example, would a student who spends more hours studying and sleeps badly perform better than one who studies less but spends enough hours sleeping? To answer these questions, there is a necessity for a broader data-oriented strategy instead of subjective methods.",
                                style={'lineHeight': '1.8', 'fontSize': '1.1rem', 'color': '#444'}
                            ),
                        ], className="mb-5"),
                       
                        html.Div([
                             html.H4("Our Narrative", className="fw-bold mb-3", style={'color': COLORS['primary']}),
                             html.P(
                                "The presented paper tells the story of modern student life: the mix of old-fashioned study and digital distractions, "
                                "the role of work-life balance on academic performance, and the significance of mental and physical health on academic success. "
                                "The raw data will be coded into a visual story on the dashboard, which guides the educational faculty.",
                                style={'lineHeight': '1.8', 'fontSize': '1.1rem', 'color': '#444'}
                            ),
                        ])
                    ],
                    lg=7, className="pe-lg-5"
                ),
                # Right Column (Goal Card)
                dbc.Col(
                    [
                        dbc.Card(
                            dbc.CardBody([
                                html.Div(html.I(className="fas fa-bullseye fa-2x", style={'color': COLORS['white']}), 
                                         style={'backgroundColor': COLORS['secondary'], 'width': '60px', 'height': '60px', 'borderRadius': '50%', 'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center', 'marginBottom': '25px', 'boxShadow': '0 10px 20px -10px rgba(26, 147, 111, 0.5)'}),
                                html.H3("Project Goal", className="fw-bold mb-3", style={'color': COLORS['dark']}),
                                html.P(
                                    "To build an interactive dashboard that presents actionable insights on the correlation between student habits and academic results, enabling educators and students to make better decisions to improve academic results.",
                                    style={'fontSize': '1.1rem', 'lineHeight': '1.6', 'color': '#555'}
                                )
                            ], className="p-4"),
                            className="styled-card h-100",
                            style={'borderTop': f'5px solid {COLORS["secondary"]} !important'}
                        ),
                    ],
                    lg=5, className="mt-4 mt-lg-0"
                ),
            ],
            className="mb-4"
        )
    ]

    # --- 2. Content for Target Audience ---
    def create_enhanced_audience_card(title, icon, color, audience_list):
         return dbc.Card(
             dbc.CardBody([
                 html.I(className=f"fas {icon} fa-3x mb-4", style={'color': color, 'opacity': 0.9}),
                 html.H5(title, className="fw-bold mb-3"),
                 html.Ul(audience_list, className="text-muted text-start d-inline-block", style={'fontSize': '0.95rem'})
             ], className="text-center p-4"), 
             className="styled-card h-100"
         )

    target_audience_content = [
        dbc.Row(
            [
                dbc.Col(create_enhanced_audience_card("Primary Audience", "fa-users", COLORS['primary'],
                        [html.Li("University Administrators"), html.Li("Academic Advisors"), html.Li("Student Support Services")]),
                    lg=4, md=6, className="mb-4"),
                dbc.Col(create_enhanced_audience_card("Secondary Audience", "fa-flask", COLORS['secondary'],
                        [html.Li("Educational Researchers"), html.Li("Policy Makers"), html.Li("Curriculum Developers")]),
                    lg=4, md=6, className="mb-4"),
                dbc.Col(create_enhanced_audience_card("Tertiary Audience", "fa-user-graduate", COLORS['tertiary'],
                        [html.Li("Students & Parents"), html.Li("Educational Tech Developers")]),
                    lg=4, md=12, className="mb-4"),
            ]
        )
    ]

    # --- 3. Content for Dataset Overview ---
    dataset_variable_data = {
        'student_id': {'range': 'Sxxxx', 'desc': 'Unique identifier for each student.', 'icon': 'fa-id-card', 'color': COLORS['accent1'], 'theme': 'academic'},
        'study_hours_per_day': {'range': '0.0 – 8.3', 'desc': 'Average number of hours spent studying daily.', 'icon': 'fa-book-open', 'color': COLORS['accent1'], 'theme': 'academic'},
        'attendance_percentage': {'range': '56.0 – 100.0', 'desc': 'Percentage of class attendance.', 'icon': 'fa-calendar-check', 'color': COLORS['accent1'], 'theme': 'academic'},
        'part_time_job': {'range': 'Yes, No', 'desc': 'Whether the student has a part-time job.', 'icon': 'fa-briefcase', 'color': COLORS['accent2'], 'theme': 'academic'},
        'parental_education_level': {'range': 'None, High School, ...', 'desc': 'Highest education level attained by parents.', 'icon': 'fa-user-tie', 'color': COLORS['accent2'], 'theme': 'academic'},
        'internet_quality': {'range': 'Poor, Average, Good', 'desc': 'Quality of student’s internet connection.', 'icon': 'fa-wifi', 'color': COLORS['accent2'], 'theme': 'academic'},
        'exam_score': {'range': '18.4 – 100.0', 'desc': 'Final academic exam score.', 'icon': 'fa-graduation-cap', 'color': COLORS['primary'], 'theme': 'academic'},
        'sleep_hours': {'range': '3.2 – 10.0', 'desc': 'Average hours of sleep per day.', 'icon': 'fa-bed', 'color': COLORS['accent3'], 'theme': 'physical'},
        'diet_quality': {'range': 'Poor, Fair, Good', 'desc': 'Self-assessed diet quality.', 'icon': 'fa-apple-alt', 'color': COLORS['accent4'], 'theme': 'physical'},
        'exercise_frequency': {'range': '0 – 6', 'desc': 'Number of times exercised per week.', 'icon': 'fa-dumbbell', 'color': COLORS['secondary'], 'theme': 'physical'},
        'age': {'range': '17-24', 'desc': 'Age of the student (in years).', 'icon': 'fa-birthday-cake', 'color': COLORS['accent5'], 'theme': 'wellbeing'},
        'gender': {'range': 'Male, Female, Other', 'desc': 'Gender identity of the student.', 'icon': 'fa-venus-mars', 'color': COLORS['accent5'], 'theme': 'wellbeing'},
        'social_media_hours': {'range': '0.5 – 7.2', 'desc': 'Daily social media usage time.', 'icon': 'fa-share-alt', 'color': COLORS['accent6'], 'theme': 'wellbeing'},
        'netflix_hours': {'range': '0.0 – 5.4', 'desc': 'Daily time spent watching Netflix.', 'icon': 'fa-tv', 'color': COLORS['accent6'], 'theme': 'wellbeing'},
        'mental_health_rating': {'range': '1 – 10', 'desc': 'Self-rated mental health score (1-10).', 'icon': 'fa-brain', 'color': COLORS['tertiary'], 'theme': 'wellbeing'},
        'extracurricular_participation': {'range': 'Yes, No', 'desc': 'Participates in extracurriculars.', 'icon': 'fa-users', 'color': COLORS['tertiary'], 'theme': 'wellbeing'}
    }

    variable_accordion = html.Div(dbc.Accordion([
        dbc.AccordionItem(dbc.Row([create_variable_flip_card(n, d['range'], d['desc'], d['icon'], d['color']) for n, d in dataset_variable_data.items() if d['theme'] == 'academic'], className="mt-3 px-2"), title="Academic & Socioeconomic Variables", item_id="item-1"),
        dbc.AccordionItem(dbc.Row([create_variable_flip_card(n, d['range'], d['desc'], d['icon'], d['color']) for n, d in dataset_variable_data.items() if d['theme'] == 'physical'], className="mt-3 px-2 theme-physical"), title="Physical Health Variables", item_id="item-2"),
        dbc.AccordionItem(dbc.Row([create_variable_flip_card(n, d['range'], d['desc'], d['icon'], d['color']) for n, d in dataset_variable_data.items() if d['theme'] == 'wellbeing'], className="mt-3 px-2 theme-wellbeing"), title="Lifestyle & Wellbeing Variables", item_id="item-3")
    ], active_item=["item-1", "item-2", "item-3"], flush=True, always_open=True, className="styled-accordion"), className="p-4 bg-white rounded-3 shadow-sm")

    dataset_content = [
        dbc.Row(
            [
                dbc.Col(
                    [
                        html.P(
                            "The dataset used in this analysis comprises comprehensive student profiles capturing academic performance, "
                            "lifestyle habits, and socioeconomic factors. Sourced from educational research repositories, this dataset "
                            "enables multidimensional analysis of factors influencing student success.",
                            className="lead mb-4", style={'color': COLORS['dark']}
                        ),
                         dbc.Row([
                            dbc.Col(html.Div([
                                html.H6("Sample Size", className="text-muted text-uppercase small fw-bold mb-2"),
                                html.H4([html.I(className="fas fa-user-friends me-2", style={'color': COLORS['primary']}), f"{len(df)} Records"], className="fw-bold")
                            ], className="p-3 bg-light rounded-3"), md=6, className="mb-3"),
                             dbc.Col(html.Div([
                                html.H6("Variables", className="text-muted text-uppercase small fw-bold mb-2"),
                                html.H4([html.I(className="fas fa-database me-2", style={'color': COLORS['secondary']}), "15+ Factors"], className="fw-bold")
                            ], className="p-3 bg-light rounded-3"), md=6, className="mb-3"),
                        ])
                    ], lg=8, className="pe-lg-5 mb-4"),
                dbc.Col(
                    dbc.Row([
                        dbc.Col(html.Div([
                            html.H3(f"{df['exam_score'].mean():.1f}%", className="stat-box-value", style={'color': COLORS['success']}),
                            html.Div("Avg Exam Score", className="stat-box-label")
                        ], className="stat-box"), width=6, className="mb-4"),
                         dbc.Col(html.Div([
                            html.H3(f"{df['study_hours_per_day'].mean():.1f}h", className="stat-box-value", style={'color': COLORS['info']}),
                            html.Div("Avg Daily Study", className="stat-box-label")
                        ], className="stat-box"), width=6, className="mb-4"),
                         dbc.Col(html.Div([
                            html.H3(f"{df['attendance_percentage'].mean():.0f}%", className="stat-box-value", style={'color': COLORS['primary']}),
                            html.Div("Avg Attendance", className="stat-box-label")
                        ], className="stat-box"), width=6),
                         dbc.Col(html.Div([
                            html.H3(f"{df['sleep_hours'].mean():.1f}h", className="stat-box-value", style={'color': COLORS['tertiary']}),
                            html.Div("Avg Sleep", className="stat-box-label")
                        ], className="stat-box"), width=6),
                    ]), lg=4),
            ], className="mb-5 align-items-center"),
        html.H5("Explore Variables", className="fw-bold mb-4", style={'color': COLORS['dark']}),
        variable_accordion
    ]

    # --- 4. Content for Analysis Framework ---
    # Static Heatmap
    static_numeric_df = df.select_dtypes(include='number')
    static_corr_matrix = static_numeric_df.corr()
    static_heatmap_fig = px.imshow(static_corr_matrix, text_auto='.2f', color_continuous_scale="RdBu_r", aspect="auto")
    static_heatmap_fig.update_layout(template="plotly_white", height=450, paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', margin=dict(l=0, r=0, t=20, b=0))

    framework_content = [
        # -- Row 1: The 4 Dimensions Cards --
        html.P("Our analysis employs EDA techniques to uncover patterns, encompassing four key dimensions:", className="lead mb-4"),
        dbc.Row([
            dbc.Col(dbc.Card(dbc.CardBody([
                html.Div(html.I(className="fas fa-check-circle", style={'color': COLORS['primary'], 'fontSize': '1.5rem'}), className="mb-3"),
                html.H6("Known vs Known", className="fw-bold", style={'color': COLORS['primary']}), 
                html.P("Confirming established relationships (e.g., Attendance → Scores)", className="small text-muted mb-0")
            ]), className="styled-card text-center h-100 p-3", style={'borderTop': f'4px solid {COLORS["primary"]} !important'}), md=6, lg=3, className="mb-4"),
            dbc.Col(dbc.Card(dbc.CardBody([
                    html.Div(html.I(className="fas fa-search-plus", style={'color': COLORS['secondary'], 'fontSize': '1.5rem'}), className="mb-3"),
                html.H6("Known vs Unknown", className="fw-bold", style={'color': COLORS['secondary']}), 
                html.P("Quantifying effects of known factors (e.g., Optimal study hours)", className="small text-muted mb-0")
            ]), className="styled-card text-center h-100 p-3", style={'borderTop': f'4px solid {COLORS["secondary"]} !important'}), md=6, lg=3, className="mb-4"),
            dbc.Col(dbc.Card(dbc.CardBody([
                    html.Div(html.I(className="fas fa-lightbulb", style={'color': COLORS['accent6'], 'fontSize': '1.5rem'}), className="mb-3"),
                html.H6("Unknown vs Known", className="fw-bold", style={'color': COLORS['accent6']}), 
                html.P("Discovering new factors (e.g., Age group social media impact)", className="small text-muted mb-0")
            ]), className="styled-card text-center h-100 p-3", style={'borderTop': f'4px solid {COLORS["accent6"]} !important'}), md=6, lg=3, className="mb-4"),
            dbc.Col(dbc.Card(dbc.CardBody([
                    html.Div(html.I(className="fas fa-project-diagram", style={'color': COLORS['tertiary'], 'fontSize': '1.5rem'}), className="mb-3"),
                html.H6("Unknown vs Unknown", className="fw-bold", style={'color': COLORS['tertiary']}), 
                html.P("Exploring interactions (e.g., Diet + Sleep + Mental Health)", className="small text-muted mb-0")
            ]), className="styled-card text-center h-100 p-3", style={'borderTop': f'4px solid {COLORS["tertiary"]} !important'}), md=6, lg=3, className="mb-4"),
        ], className="mb-5"),

        # -- Row 2: Detailed Analysis (Left Col: Text, Right Col: Heatmap) --
        dbc.Row([
             dbc.Col([
                # 1. Data Quality
                html.H5([html.I(className="fas fa-database me-2", style={'color': COLORS['success']}), "1. Data Quality"], className="fw-bold mt-2"),
                html.Ul([
                    html.Li("No missing values (after cleaning 'parental_education')."),
                    html.Li("No duplicate entries."),
                    html.Li("All variables fall within realistic valid ranges."),
                ], className="text-muted mb-4"),

                # 2. Academic Performance Overview
                html.H5([html.I(className="fas fa-graduation-cap me-2", style={'color': COLORS['primary']}), "2. Academic Performance Overview"], className="fw-bold"),
                html.Ul([
                     html.Li(["Exam scores: Mean ", html.Strong("65.2%"), ", Median ", html.Strong("66.0%"), ", Std Dev 15.8%."]),
                     html.Li("Range: 18.4% to 100%."),
                     html.Li("Slightly left-skewed distribution; most scores between 50–80%."),
                ], className="text-muted mb-4"),

                # 3. Key Correlations
                html.H5([html.I(className="fas fa-link me-2", style={'color': COLORS['info']}), "3. Key Correlations"], className="fw-bold"),
                dbc.Table([
                    html.Thead(html.Tr([html.Th("Relationship"), html.Th("Direction"), html.Th("r Value")])),
                    html.Tbody([
                        html.Tr([html.Td("Attendance ↔ Exam Score"), html.Td([html.I(className="fas fa-arrow-up text-success me-1"), " Positive"]), html.Td(html.Strong("0.68"))]),
                        html.Tr([html.Td("Study Hours ↔ Exam Score"), html.Td([html.I(className="fas fa-arrow-up text-success me-1"), " Positive"]), html.Td("0.42")]),
                        html.Tr([html.Td("Social Media ↔ Exam Score"), html.Td([html.I(className="fas fa-arrow-down text-danger me-1"), " Negative"]), html.Td("-0.38")]),
                        html.Tr([html.Td("Sleep ↔ Mental Health"), html.Td([html.I(className="fas fa-arrow-up text-success me-1"), " Positive"]), html.Td("0.45")]),
                    ])
                ], bordered=False, hover=True, striped=True, size="sm", className="mb-4 shadow-sm", style={'borderRadius': '10px', 'overflow': 'hidden'}),
                 
                 dbc.Row([
                     # 4. Notable Patterns
                     dbc.Col(dbc.Card(dbc.CardBody([
                         html.H6([html.I(className="fas fa-lightbulb me-2", style={'color': COLORS['accent6']}), "4. Notable Patterns"], className="fw-bold"),
                         html.Ul([
                            html.Li("Attendance is the strongest predictor."),
                            html.Li("Study hours >8/day show diminishing returns."),
                            html.Li("Social media >4hrs/day hurts scores."),
                        ], className="small text-muted ps-3 mb-0")
                     ]), className="styled-card h-100 py-3"), md=6),
                     # 5. Limitations
                      dbc.Col(dbc.Card(dbc.CardBody([
                         html.H6([html.I(className="fas fa-exclamation-triangle me-2", style={'color': COLORS['warning']}), "5. Limitations"], className="fw-bold"),
                         html.Ul([
                            html.Li("No cultural context."),
                            html.Li("Snapshot data only."),
                            html.Li("Limited to exam scores."),
                        ], className="small text-muted ps-3 mb-0")
                     ]), className="styled-card h-100 py-3"), md=6),
                 ])
                 
             ], lg=6, className="pe-lg-5"),

             # Right Column: Heatmap & Findings
             dbc.Col([
                 dbc.Card([
                     dbc.CardHeader("Correlation Heatmap", className="bg-white fw-bold border-bottom-0 pt-4 px-4", style={'color': COLORS['dark']}),
                     dbc.CardBody(dcc.Graph(
                            figure=static_heatmap_fig,
                            style={
                                'height': '350px',
                                'width': '100%',
                            }), className="styled-card h"), 
                 ]),
             ], lg=6, md=12, className="mt-4"),
        ])
    ]

    # --- FINAL PAGE ASSEMBLY ---
    return html.Div([
        hero_section,
        html.Div(dbc.Container([html.Div(id="intro-case"), html.H2("1. Introduction to the Case", className="section-header"), *intro_case_content], fluid=False, className="py-5"), className="intro-section-white"),
        html.Div(dbc.Container([html.H2("2. Target Audience", className="section-header"), *target_audience_content], fluid=False, className="py-5"), className="intro-band-blue"),
        html.Div(dbc.Container([html.H2("3. Dataset Overview", className="section-header"), *dataset_content], fluid=False, className="py-5"), className="intro-section-white"),
        html.Div(dbc.Container([html.H2("4. Analysis Framework", className="section-header"), *framework_content], fluid=False, className="py-5"), className="intro-band-blue"),
        
        html.Div(dbc.Container(dbc.Row(dbc.Col(html.Div([
            html.H2("Ready to dive into the data?", className="fw-black mb-4"),
            dcc.Link(dbc.Button("Launch Interactive Dashboard", color="primary", size="lg", className="px-5 py-3 fw-bold rounded-pill"), href="/dashboard")
        ], className="text-center py-5"), width=12)), fluid=False), className="intro-section-green")
    ])

### 3. About Us

In [7]:
def create_about_us_page():
    """
    Creates the 'About Us' page layout for a Dash application.
    """
    return dbc.Container(
        [
            # --- Page Header ---
            dbc.Row([dbc.Col([html.H1("Meet the Minds", className="text-center fw-black display-4 mb-3", style={'color': COLORS['primary']}),
                            html.P("The dedicated team behind this data visualisation project for ICT305.", className="text-center lead text-muted mb-5", style={'maxWidth': '700px', 'margin': '0 auto'}
                            )])],className="mt-5"),

            # --- Team Row 1 ---
            dbc.Row([
                    # --- Wadiqa Baig ---
                    dbc.Col(dbc.Card(dbc.CardBody([html.Div(html.I(className="fas fa-user-astronaut fa-3x", style={'color': 'white'}),style={
                                        'backgroundColor': COLORS['accent1'],
                                        'width': '100px',
                                        'height': '100px',
                                        'borderRadius': '50%',
                                        'display': 'flex',
                                        'alignItems': 'center',
                                        'justifyContent': 'center',
                                        'margin': '0 auto 25px auto',
                                        'boxShadow': f'0 10px 25px -8px {COLORS["accent1"]}'
                                    }
                                ),
                                html.H4("Wadiqa Baig", className="fw-bold mb-2", style={'color': COLORS['dark']}),
                                dbc.Badge("UX/UI Designer", color="light", text_color="dark", className="px-3 py-2")
                            ]),
                            className="styled-card text-center h-100 py-4"
                        ),
                        lg=4,
                        className="mb-4"
                    ),

                    # --- Areeba Shoaib ---
                    dbc.Col(
                        dbc.Card(
                            dbc.CardBody([
                                html.Div(
                                    html.I(className="fas fa-paint-brush fa-3x", style={'color': 'white'}),
                                    style={
                                        'backgroundColor': COLORS['secondary'],
                                        'width': '100px',
                                        'height': '100px',
                                        'borderRadius': '50%',
                                        'display': 'flex',
                                        'alignItems': 'center',
                                        'justifyContent': 'center',
                                        'margin': '0 auto 25px auto',
                                        'boxShadow': f'0 10px 25px -8px {COLORS["secondary"]}'
                                    }
                                ),
                                html.H4("Areeba Shoaib", className="fw-bold mb-2", style={'color': COLORS['dark']}),
                                dbc.Badge("Visualisation Expert", color="light", text_color="dark", className="px-3 py-2")
                            ]),
                            className="styled-card text-center h-100 py-4"
                        ),
                        lg=4,
                        className="mb-4"
                    ),

                    # --- Warda Baig ---
                    dbc.Col(
                        dbc.Card(
                            dbc.CardBody([
                                html.Div(
                                    html.I(className="fas fa-microscope fa-3x", style={'color': 'white'}),
                                    style={
                                        'backgroundColor': COLORS['tertiary'],
                                        'width': '100px',
                                        'height': '100px',
                                        'borderRadius': '50%',
                                        'display': 'flex',
                                        'alignItems': 'center',
                                        'justifyContent': 'center',
                                        'margin': '0 auto 25px auto',
                                        'boxShadow': f'0 10px 25px -8px {COLORS["tertiary"]}'
                                    }
                                ),
                                html.H4("Warda Baig", className="fw-bold mb-2", style={'color': COLORS['dark']}),
                                dbc.Badge("Strategic Lead", color="light", text_color="dark", className="px-3 py-2")
                            ]),
                            className="styled-card text-center h-100 py-4"
                        ),
                        lg=4,
                        className="mb-4"
                    )
                ],
                justify="center"
            ),

            # --- Team Row 2 ---
            dbc.Row(
                [
                    # --- Lidya Tesfay Adal ---
                    dbc.Col(
                        dbc.Card(
                            dbc.CardBody([
                                html.Div(
                                    html.I(className="fas fa-pen-nib fa-3x", style={'color': 'white'}),
                                    style={
                                        'backgroundColor': COLORS['accent6'],
                                        'width': '100px',
                                        'height': '100px',
                                        'borderRadius': '50%',
                                        'display': 'flex',
                                        'alignItems': 'center',
                                        'justifyContent': 'center',
                                        'margin': '0 auto 25px auto',
                                        'boxShadow': f'0 10px 25px -8px {COLORS["accent6"]}'
                                    }
                                ),
                                html.H4("Lidya Tesfay Adal", className="fw-bold mb-2", style={'color': COLORS['dark']}),
                                dbc.Badge("Data Analyst", color="light", text_color="dark", className="px-3 py-2")
                            ]),
                            className="styled-card text-center h-100 py-4"
                        ),
                        lg=4,
                        className="mb-4"
                    ),

                    # --- Hiba Zubairi ---
                    dbc.Col(
                        dbc.Card(
                            dbc.CardBody([
                                html.Div(
                                    html.I(className="fas fa-chess-queen fa-3x", style={'color': 'white'}),
                                    style={
                                        'backgroundColor': COLORS['accent5'],
                                        'width': '100px',
                                        'height': '100px',
                                        'borderRadius': '50%',
                                        'display': 'flex',
                                        'alignItems': 'center',
                                        'justifyContent': 'center',
                                        'margin': '0 auto 25px auto',
                                        'boxShadow': f'0 10px 25px -8px {COLORS["accent5"]}'
                                    }
                                ),
                                html.H4("Hiba Zubairi", className="fw-bold mb-2", style={'color': COLORS['dark']}),
                                dbc.Badge("Researcher Lead", color="light", text_color="dark", className="px-3 py-2")
                            ]),
                            className="styled-card text-center h-100 py-4"
                        ),
                        lg=4,
                        className="mb-4"
                    )
                ],
                justify="center",
                className="mb-5"
            ),

            # --- Footer Info ---
            dbc.Row(
                [
                    dbc.Col(
                        dbc.Card(
                            dbc.CardBody([
                                html.H4(
                                    [html.I(className="fas fa-university me-3"), "Murdoch University Dubai"],
                                    className="fw-bold",
                                    style={'color': COLORS['primary']}
                                ),
                                html.P(
                                    "ICT305: Data Visualisation and Simulation | Group Project 2025",
                                    className="mb-0 text-muted"
                                )
                            ]),
                            className="styled-card",
                            style={'borderLeft': f"8px solid {COLORS['secondary']}"}
                        ),
                        lg=8,
                        className="mx-auto"
                    )
                ],
                className="mb-5"
            )
        ],
        fluid=True,
        style={'maxWidth': '1200px', 'paddingBottom': '60px'}
    )

### 4. Acknowledgment

In [8]:
def create_acknowledgments_page():
    # --- Header Section ---
    header = dbc.Row([
        dbc.Col([
            html.H1("Acknowledgments", className="text-center fw-black display-4 mb-3", style={'color': COLORS['primary']}),
            html.P("This project was made possible by referencing the work of others and using publicly available resources.", 
                   className="text-center lead text-muted mb-5", style={'maxWidth': '700px', 'margin': '0 auto'})
        ], lg=8, className="mx-auto")
    ], className="mt-5")

    # --- Academic References Section (Wide Card) ---
    academic_section = dbc.Card(dbc.CardBody([
        html.H3([html.I(className="fas fa-book-reader me-3", style={'color': COLORS['primary']}), "Academic References"], 
                className="fw-bold mb-4", style={'color': COLORS['dark']}),
        html.Ul([
            html.Li([
                html.Strong("Lakhani, K. (2024)."), " Sleep Patterns and Alertness in Class. ",
                html.I("The International Journal of Indian Psychology, 12"), "(4). ",
                html.A([html.I(className="fas fa-external-link-alt ms-1 small"), " View Article"], 
                       href="https://ijip.in/articles/sleep-patterns-and-alertness/", target="_blank", 
                       className="ms-1", style={'color': COLORS['primary'], 'textDecoration': 'none', 'fontWeight': '600'})
            ], className="mb-3"),
            html.Li([
                html.Strong("Mary, B. J. (2024, December 30)."), " ",
                html.I("A Quantitative Study of the Impact of Daily Study Hours on Academic Performance Among High School Students"),
                ". ResearchGate. ",
                html.A([html.I(className="fas fa-external-link-alt ms-1 small"), " View Publication"], 
                       href="https://www.researchgate.net/publication/387534719_A_Quantitative_Study_of_the_Impact_of_Daily_Study_Hours_on_Academic_Performance_Among_High_School_Students", 
                       target="_blank", className="ms-1", style={'color': COLORS['primary'], 'textDecoration': 'none', 'fontWeight': '600'})
            ], className="mb-3")
        ], className="text-muted ps-4", style={'fontSize': '1.05rem', 'lineHeight': '1.7'})
    ]), className="styled-card p-4 mb-5", style={'borderLeft': f"6px solid {COLORS['primary']}"})

    # --- Tools & Resources Section (3-Column Grid) ---
    resources_row = dbc.Row([
        # 1. Dataset Card
        dbc.Col(dbc.Card(dbc.CardBody([
            html.Div(html.I(className="fas fa-database fa-2x", style={'color': 'white'}), 
                     style={'backgroundColor': COLORS['secondary'], 'width': '70px', 'height': '70px', 'borderRadius': '50%', 
                            'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center', 'margin': '0 auto 25px auto', 
                            'boxShadow': f'0 10px 20px -10px {COLORS["secondary"]}'}),
            html.H5("Dataset Source", className="fw-bold mb-3", style={'color': COLORS['dark']}),
            html.P([
                "Nath, J. (2025).", html.Br(),
                html.I("Student Habits vs Academic Performance."), html.Br(),
                "Kaggle.com."
            ], className="text-muted mb-3 small"),
            dbc.Button([html.I(className="fas fa-table me-2"), "View Dataset"], 
                       href="https://www.kaggle.com/datasets/jayaantanaath/student-habits-vs-academic-performance/data", 
                       target="_blank", color="light", className="mt-auto", 
                       style={'color': COLORS['secondary'], 'fontWeight': '600', 'fontSize': '0.9rem'})
        ]), className="styled-card h-100 text-center p-4 d-flex flex-column"), lg=4, md=6, className="mb-4"),

        # 2. Inspiration Card
        dbc.Col(dbc.Card(dbc.CardBody([
            html.Div(html.I(className="fab fa-youtube fa-2x", style={'color': 'white'}), 
                     style={'backgroundColor': COLORS['tertiary'], 'width': '70px', 'height': '70px', 'borderRadius': '50%', 
                            'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center', 'margin': '0 auto 25px auto', 
                            'boxShadow': f'0 10px 20px -10px {COLORS["tertiary"]}'}),
            html.H5("Dashboard Inspiration", className="fw-bold mb-3", style={'color': COLORS['dark']}),
            html.P([
                "Layout concepts adapted from", html.Br(),
                html.Strong("'Charming Data'"), " on YouTube."
            ], className="text-muted mb-4 small"),
             dbc.Button([html.I(className="fab fa-youtube me-2"), "Watch Tutorial"], 
                       href="https://youtu.be/PdPPR6co7nY?si=X1WOspKiiEC0kwaO", 
                       target="_blank", color="light", className="mt-auto", 
                       style={'color': COLORS['tertiary'], 'fontWeight': '600', 'fontSize': '0.9rem'})
        ]), className="styled-card h-100 text-center p-4 d-flex flex-column"), lg=4, md=6, className="mb-4"),

        # 3. Logo Card
        dbc.Col(dbc.Card(dbc.CardBody([
            html.Div(html.I(className="fas fa-paint-brush fa-2x", style={'color': 'white'}), 
                     style={'backgroundColor': COLORS['accent2'], 'width': '70px', 'height': '70px', 'borderRadius': '50%', 
                            'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center', 'margin': '0 auto 25px auto', 
                            'boxShadow': f'0 10px 20px -10px {COLORS["accent2"]}'}),
            html.H5("Logo Design", className="fw-bold mb-3", style={'color': COLORS['dark']}),
            html.P([
                "The 'DataTales' logo was designed with generative assistance from ", 
                html.Strong("Google's Gemini 2.5 Pro"), "."
            ], className="text-muted mb-3 small"),
             dbc.Badge("AI Assisted", color="light", text_color="dark", className="px-3 py-2 mt-auto align-self-center")
        ]), className="styled-card h-100 text-center p-4 d-flex flex-column"), lg=4, md=12, className="mb-4"),
    ])

    # --- Final Page Assembly ---
    return dbc.Container([
        header,
        dbc.Row(dbc.Col(academic_section, lg=10, className="mx-auto")),
        dbc.Row(dbc.Col(resources_row, lg=10, className="mx-auto")),
    ], fluid=True, style={'paddingBottom': '80px', 'maxWidth': '1200px'})

## SECTION 6: DASHBOARD TABS & LAYOUT

### Theme 1: Academic factors

In [9]:
def create_academic_factors_tab():
    """Holds graphs for H1 and H2."""
    return html.Div([
        dbc.Row([
            dbc.Col(dbc.Card(dbc.CardBody([
                html.H4("Attendance Impact Analysis", style={'color': COLORS['accent1'], 'fontWeight': '600'}),
                html.P("Higher attendance percentage groups consistently shows higher median and overall exam scores.", className="text-muted mb-4", style={'fontStyle': 'italic'}),
                dcc.Graph(id="attendance-line-chart", style={'height': '500px'}) 
            ]), className="styled-card"), lg=12), 
        ]),
        
        dbc.Row([
            dbc.Col(dbc.Card(dbc.CardBody([
                html.H4("Study Efficiency Analysis", style={'color': COLORS['accent2'], 'fontWeight': '600'}),
                html.P("Students who spend 6–8 hours of study per day come into the category of top scoring results, appearing optimal for performance.", className="text-muted mb-4", style={'fontStyle': 'italic'}),
                dcc.Graph(id="study-hours-bar-chart", style={'height': '500px'}) 
            ]), className="styled-card"), lg=12), 
        ], className="mt-4"),
    ])

### Theme 2: Lifestyle wellbeing Factors

In [10]:
def create_lifestyle_wellbeing_tab():
    """Holds graphs for H3, H4, H6, H7, H9."""
    return html.Div([
        dbc.Row([
            dbc.Col(dbc.Card(dbc.CardBody([
                html.H4("Sleep & Mental Health Analysis", style={'color': COLORS['accent3'], 'fontWeight': '600'}),
                html.P("Although the chart title suggests 7–9 hours is optimal, the less than 7 hours group actually reported a just barely higher average mental health rating (5.5 vs. 5.4).", className="text-muted mb-4", style={'fontStyle': 'italic'}),
                dcc.Graph(id="sleep-mental-health-chart", style={'height': '500px'}) # H4
            ]), className="styled-card"), lg=6, className="mb-4"),
            
            dbc.Col(dbc.Card(dbc.CardBody([
                html.H3("Social Media & Exam Score", style={'color': COLORS['accent5'], 'fontWeight': '600'}),
                html.P("Students using 0–2 hours of social media have the highest average exam score, depicting a negative correlation with increased use.", className="text-muted mb-4", style={'fontStyle': 'italic'}),
                dcc.Graph(id="social-media-exam-chart", style={'height': '500px'}) # H3
            ]), className="styled-card"), lg=6, className="mb-4"),
        ]),
        
        dbc.Row([
             dbc.Col(dbc.Card(dbc.CardBody([
                html.H4("Extracurricular Impact Analysis", style={'color': COLORS['tertiary'], 'fontWeight': '600'}),
                html.P("While fewer students participate overall, the data shows participation offers no improvement in well-being.", className="text-muted mb-4", style={'fontStyle': 'italic'}),
                dcc.Graph(id="extracurricular-mental-chart", style={'height': '500px'}) # H9
            ]), className="styled-card"), lg=12, className="mb-4"),

            dbc.Col(dbc.Card(dbc.CardBody([
                html.H4("Diet Quality Impact Analysis", style={'color': COLORS['secondary'], 'fontWeight': '600'}),
                # Use triple quotes """ for text containing "double quotes"
                html.P(
                    """ "Good" diet correlates with top scores, the "Fair" diet group also contains a significant 29% of students achieving the highest grades (80–100), which indicates that other factors impact greatly.""", 
                    className="text-muted mb-4", style={'fontStyle': 'italic'}
                ),
                dcc.Graph(id="diet-exam-chart", style={'height': '500px'}) 
            ]), className="styled-card"), lg=12, className="mb-4"),
        ]),
        
         dbc.Row([
            dbc.Col(dbc.Card(dbc.CardBody([
                html.H4("Age & Social Media Analysis", style={'color': COLORS['accent6'], 'fontWeight': '600'}),
                html.P("Social media use is common across university ages, peaking slightly around 18 years old.", className="text-muted mb-4", style={'fontStyle': 'italic'}),
                dcc.Graph(id="age-social-media-chart", style={'height': '500px'}) # H7
            ]), className="styled-card"), lg=12, className="mb-4"), 
        ]),
    ])

### Theme 3: Socioeconomic Factors

In [11]:
def create_socioeconomic_factors_tab():
    """Holds graphs for H8 and H10."""
    return html.Div([
        dbc.Row([
            dbc.Col(dbc.Card(dbc.CardBody([
                html.H4("Part-Time Work Impact Analysis", style={'color': COLORS['accent1'], 'fontWeight': '600'}),
                html.P("Students without a part-time job show a slight, but observable, advantage in median exam scores.", className="text-muted mb-4", style={'fontStyle': 'italic'}),
                dcc.Graph(id="part-time-job-chart", style={'height': '350px'})
            ]), className="styled-card"), lg=12, className="mb-4"), 
        ]),
        
        dbc.Row([
            dbc.Col(dbc.Card(dbc.CardBody([
                html.H4("Internet Quality Impact Analysis", style={'color': COLORS['accent2'], 'fontWeight': '600'}),
                html.P("No link between internet quality with exam scores", className="text-muted mb-4", style={'fontStyle': 'italic'}),
                dcc.Graph(id="internet-quality-chart", style={'height': '500px'})
            ]), className="styled-card"), lg=12, className="mb-4"),
        ]),
    ])

## SECTION 7: MAIN DASHBOARD PAGE

In [12]:
def create_dashboard_page():
    """Creates the main interactive dashboard with sticky sidebar and modern header."""
    
    # 1. Sticky Sidebar
    sidebar = html.Div(
        [
            dbc.Card(dbc.CardBody([
                html.H4([html.I(className="fas fa-filter me-2"), "Filters"], className="mb-4", style={'color': COLORS['primary']}),
                dbc.Label(["Gender ", html.I(className="fas fa-venus-mars small text-muted")]),
                dcc.Dropdown(id="gender-filter", options=[{"label": g, "value": g} for g in sorted(df["gender"].unique())], multi=True, placeholder="All Genders", style={'marginBottom': '20px'}),
                dbc.Label(["Part-time Job ", html.I(className="fas fa-briefcase small text-muted")]),
                dcc.Dropdown(id="job-filter", options=[{"label": j, "value": j} for j in sorted(df["part_time_job"].unique())], multi=True, placeholder="All Status", style={'marginBottom': '20px'}),
                dbc.Label(["Age Range ", html.I(className="fas fa-birthday-cake small text-muted")]),
                dcc.RangeSlider(id="age-slider", min=int(df["age"].min()), max=int(df["age"].max()), step=1, value=[int(df["age"].min()), int(df["age"].max())], marks={int(a): str(a) for a in sorted(df["age"].unique())}, tooltip={"placement": "bottom", "always_visible": True}),
            ]), className="styled-card"),
        ],
        className="sticky-sidebar"
    )
    
    # 2. Main Content Area
    main_content = html.Div(
        [
            html.Div([
                dbc.Row([
                    # Left side: Title
                    dbc.Col([
                        html.H1("Analytics Dashboard", className="mb-1", style={'fontWeight': 900, 'color': COLORS['dark'], 'letterSpacing': '-1px'}),
                        html.P("Explore dynamic relationships between lifestyle choices and academic outcomes.", className="text-muted mb-0", style={'fontSize': '1.1rem'})
                    ], lg=8, md=7),
                ], className="align-items-center")
            ], className="mb-5"),
            
            # Stats Row (Populated by Callback)
            dbc.Row(id="stat-card-container", className="mb-4"),
            
            # Tabs
            html.Div(
                [
                    html.Div([html.I(className="fas fa-graduation-cap me-2"), "Academic Factors"], id='tab-academic', n_clicks=0, className='custom-tab custom-tab--active'),
                    html.Div([html.I(className="fas fa-heartbeat me-2"), "Lifestyle & Wellbeing"], id='tab-lifestyle', n_clicks=0, className='custom-tab'),
                    html.Div([html.I(className="fas fa-globe me-2"), "Socioeconomic factors"], id='tab-socioeconomic', n_clicks=0, className='custom-tab'),
                ],
                className='tab-navigation-container'
            ),
            
            # Content Wrappers
            html.Div([
                html.Div(id='academic-content-wrapper', children=[create_academic_factors_tab()], style={'display': 'block'}),
                html.Div(id='lifestyle-content-wrapper', children=[create_lifestyle_wellbeing_tab()], style={'display': 'none'}),
                html.Div(id='socioeconomic-content-wrapper', children=[create_socioeconomic_factors_tab()], style={'display': 'none'})
            ]),
        ]
    )
    
    return dbc.Container(
        dbc.Row(
            [
                dbc.Col(sidebar, lg=3, md=4, className="mb-4"), 
                dbc.Col(main_content, lg=9, md=8),
            ],
            className="mt-4"
        ),
        fluid=True,
        style={'paddingBottom': '40px', 'maxWidth': '1600px'}
    )

## SECTION 8: APP LAYOUT & CORE CALLBACKS

In [13]:
app.layout = html.Div([dcc.Location(id='url', refresh=False), create_navbar(), html.Div(id='page-content')], style={'minHeight': '100vh', 'backgroundColor': '#F0F7F4'})
@app.callback(Output('page-content', 'children'), [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/dashboard': return create_dashboard_page()
    elif pathname == '/introduction': return create_project_introduction()
    elif pathname == '/about-us': return create_about_us_page()
    elif pathname == '/acknowledgments': return create_acknowledgments_page()
    else: return create_landing_page()

@app.callback([Output(f'nav-{i}', 'className') for i in ['home', 'intro', 'dash', 'about', 'ack']], [Input('url', 'pathname')])
def update_active_links(pathname):
    paths = ['/', '/introduction', '/dashboard', '/about-us', '/acknowledgments']
    return [f"nav-link ms-3 {'nav-link--active' if pathname == p else ''}" for p in paths]

@app.callback([Output('academic-content-wrapper', 'style'), Output('lifestyle-content-wrapper', 'style'), Output('socioeconomic-content-wrapper', 'style'), Output('tab-academic', 'className'), Output('tab-lifestyle', 'className'), Output('tab-socioeconomic', 'className')], [Input('tab-academic', 'n_clicks'), Input('tab-lifestyle', 'n_clicks'), Input('tab-socioeconomic', 'n_clicks')])
def switch_tabs(n1, n2, n3):
    ctx = callback_context
    button_id = ctx.triggered[0]['prop_id'].split('.')[0] if ctx.triggered else 'tab-academic'
    return ({'display': 'block', 'animation': 'fadeIn 0.5s'} if button_id == 'tab-academic' else {'display': 'none'}, {'display': 'block', 'animation': 'fadeIn 0.5s'} if button_id == 'tab-lifestyle' else {'display': 'none'}, {'display': 'block', 'animation': 'fadeIn 0.5s'} if button_id == 'tab-socioeconomic' else {'display': 'none'}, 'custom-tab custom-tab--active' if button_id == 'tab-academic' else 'custom-tab', 'custom-tab custom-tab--active' if button_id == 'tab-lifestyle' else 'custom-tab', 'custom-tab custom-tab--active' if button_id == 'tab-socioeconomic' else 'custom-tab')

## SECTION 9: GRAPH & DATA CALLBACKS

In [14]:
def get_filtered_data(genders, age_range, job_status):
    dff = df.copy()
    if genders: 
        dff = dff[dff["gender"].isin(genders)]
    dff = dff[(dff["age"] >= age_range[0]) & (dff["age"] <= age_range[1])]
    if job_status: 
        dff = dff[dff["part_time_job"].isin(job_status)]
    return dff

@app.callback(
    Output("stat-card-container", "children"),
    [Input("gender-filter", "value"),
     Input("age-slider", "value"),
     Input("job-filter", "value")]
)
def update_stat_cards(genders, age_range, job_status):
    dff = get_filtered_data(genders, age_range, job_status)
    
    if dff.empty:
        total_students, avg_score, avg_study, avg_attendance = "0", "N/A", "N/A", "N/A"
    else:
        total_students = f"{len(dff)}"
        avg_score = f"{dff['exam_score'].mean():.1f}"
        avg_study = f"{dff['study_hours_per_day'].mean():.1f}h"
        avg_attendance = f"{dff['attendance_percentage'].mean():.1f}%"
        
    c1 = dbc.Col(create_gradient_stat_card(total_students, "Filtered Students", "fa-users", "gradient-primary"), lg=3, md=6, sm=6)
    c2 = dbc.Col(create_gradient_stat_card(avg_score, "Avg Exam Score", "fa-star", "gradient-success"), lg=3, md=6, sm=6)
    c3 = dbc.Col(create_gradient_stat_card(avg_study, "Daily Study", "fa-clock", "gradient-info"), lg=3, md=6, sm=6)
    c4 = dbc.Col(create_gradient_stat_card(avg_attendance, "Avg Attendance", "fa-calendar-check", "gradient-tertiary"), lg=3, md=6, sm=6)
    
    return [c1, c2, c3, c4]

@app.callback(
    Output("heatmap", "figure"), 
    [Input("gender-filter", "value"),
     Input("age-slider", "value"),
     Input("job-filter", "value")]
)
def update_heatmap(genders, age_range, job_status):
    dff = get_filtered_data(genders, age_range, job_status)
    numeric_df = dff.select_dtypes(include='number')
    correlation_matrix = numeric_df.corr()
    heatmap_fig = px.imshow(
        correlation_matrix, text_auto='.2f', color_continuous_scale="RdBu_r",
        aspect="auto"
    )
    heatmap_fig.update_layout(template="plotly_white", height=500, paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
    return heatmap_fig

@app.callback(
    [Output("attendance-line-chart", "figure"),
     Output("study-hours-bar-chart", "figure")],
    [Input("gender-filter", "value"),
     Input("age-slider", "value"),
     Input("job-filter", "value")]
)
def update_academic_charts(genders, age_range, job_status):
    dff = get_filtered_data(genders, age_range, job_status)
    if dff.empty:
        return go.Figure(), go.Figure()
    
    # H1
    dff_h1 = dff.copy()
    bins_h1 = np.arange(50, 101, 10) 
    labels_h1 = [f"{bins_h1[i]}-{bins_h1[i+1]}%" for i in range(len(bins_h1)-1)]
    dff_h1["attendance_bin"] = pd.cut(
        dff_h1["attendance_percentage"], 
        bins=bins_h1,
        labels=labels_h1,
        include_lowest=True,
        right=True
    )
    attendance_fig = px.box(
        dff_h1.sort_values("attendance_bin"),
        x="attendance_bin",
        y="exam_score",
        color="attendance_bin",
        labels={
            "attendance_bin": "Attendance Percentage Group",
            "exam_score": "Exam Score"
        }
    )
    attendance_fig.update_layout(template="plotly_white", height=500, paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', showlegend=False)
    
    # H2
    bins_h2 = [0, 2, 4, 6, 8, 10]
    labels_h2 = ["0–2", "2–4", "4–6", "6–8", ">8"]
    dff["study_hour_range"] = pd.cut(dff["study_hours_per_day"], bins=bins_h2, labels=labels_h2, include_lowest=True)
    dff["study_hour_range"] = pd.Categorical(dff["study_hour_range"], categories=labels_h2, ordered=True)
    study_fig = go.Figure()
    for group in labels_h2:
        group_data = dff[dff["study_hour_range"] == group]["exam_score"]
        if len(group_data) > 0:
            study_fig.add_trace(go.Violin(
                x=group_data, y=[group] * len(group_data),
                line_color="black",
                fillcolor="lightblue" if group != "6–8" else "lightgreen",
                opacity=0.7, orientation="h", side="positive",
                width=0.9, meanline_visible=True,
                name=group, showlegend=False
            ))
    study_fig.update_layout(
        xaxis_title="Exam Score", yaxis_title="Study Hour Range",
        violingap=0.0, violinmode="overlay",
        template="plotly_white", height=500,
        paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)'
    )
    
    return attendance_fig, study_fig

@app.callback(
 [Output("sleep-mental-health-chart", "figure"),
  Output("diet-exam-chart", "figure")],
 [Input("gender-filter", "value"),
  Input("age-slider", "value"),
  Input("job-filter", "value")]
)
def update_physical_charts(genders, age_range, job_status):
    dff = get_filtered_data(genders, age_range, job_status)
    if dff.empty:
        return go.Figure(), go.Figure()

    # H4
    bins_h4 = [0, 7, 9, 24]
    labels_h4 = ['<7 hrs', '7–9 hrs', '>9 hrs']
    dff['sleep_group'] = pd.cut(dff['sleep_hours'], bins=bins_h4, labels=labels_h4, include_lowest=True)
    sleep_summary = dff.groupby('sleep_group', observed=True)['mental_health_rating'].agg(['mean', 'sem', 'count']).reset_index()
    sleep_summary = sleep_summary.sort_values('mean', ascending=False)
    colors_h4 = {'<7 hrs': '#ef5350', '7–9 hrs': '#66bb6a', '>9 hrs': '#42a5f5'}
    sleep_fig = go.Figure()
    for group in sleep_summary['sleep_group']:
        group_data = sleep_summary[sleep_summary['sleep_group'] == group]
        sleep_fig.add_trace(go.Bar(
            x=[group], y=group_data['mean'],
            marker_color=colors_h4.get(group, '#95A5A6'),
            width=0.1, showlegend=False, hoverinfo='skip'
        ))
    sleep_fig.add_trace(go.Scatter(
        x=sleep_summary['sleep_group'], y=sleep_summary['mean'],
        mode='markers+text',
        marker=dict(
            size=28,
            color=[colors_h4.get(g, '#95A5A6') for g in sleep_summary['sleep_group']],
            line=dict(width=2, color='white'), opacity=0.95
        ),
        text=[f"{v:.1f}" for v in sleep_summary['mean']],
        textposition='middle center',
        textfont=dict(color='white', size=13),
        name='Avg Mental Health Rating',
        hovertemplate="<b>%{x}</b><br>Avg Rating: %{y:.2f}<extra></extra>"
    ))
    sleep_fig.update_layout(
        xaxis_title='Sleep Duration Group', yaxis_title='Average Mental Health Rating (1–10)',
        template='plotly_white', title_x=0.5,
        yaxis=dict(range=[0, 10], dtick=1),
        plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)',
        height=500, showlegend=False
    )
    
    # H6
    bins_h6 = [0, 49, 59, 69, 79, 100]
    labels_h6 = ['F (0-49)', 'P (50-59)', 'C (60-69)', 'D (70-79)', 'HD (80-100)']
    dff['exam_score_binned'] = pd.cut(
        dff['exam_score'], bins=bins_h6, labels=labels_h6,
        right=True, include_lowest=True
    ).astype(str)
    sunburst_data = dff.groupby(['diet_quality', 'exam_score_binned'], observed=True).agg(
        student_count=('student_id', 'size')
    ).reset_index()
    color_map_diet = {'Excellent': '#27AE60', 'Good': '#2ECC71', 'Fair': '#F39C12', 'Poor': '#E74C3C', '(?)': '#95A5A6'}
    diet_fig = px.sunburst(
        sunburst_data,
        path=[px.Constant("All Students"), 'diet_quality', 'exam_score_binned'],
        values='student_count',
        color='diet_quality',
        color_discrete_map=color_map_diet,
        hover_data={'student_count': True}
    )
    diet_fig.update_layout(
        template='plotly_white', title_x=0.5,
        margin=dict(t=50, l=25, r=25, b=25), height=500,
        paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)'
    )
    diet_fig.update_traces(
        insidetextfont=dict(color='#333333'),
        textinfo="label+percent parent",
        hovertemplate="<b>%{label}</b><br>Diet Quality: %{parent}<br>Number of Students: %{value}<br>Percentage: %{percentParent:.1%}<extra></extra>"
    )
    
    return sleep_fig, diet_fig

@app.callback(
    [Output("social-media-exam-chart", "figure"),
     Output("age-social-media-chart", "figure"),
     Output("extracurricular-mental-chart", "figure")],
    [Input("gender-filter", "value"),
     Input("age-slider", "value"),
     Input("job-filter", "value")]
)
def update_mental_charts(genders, age_range, job_status):
    dff = get_filtered_data(genders, age_range, job_status)
    if dff.empty:
        return go.Figure(), go.Figure(), go.Figure()
    
    # H3
    bins_h3 = [0, 2, 4, 6, np.inf] 
    labels_h3 = ['0–2 hrs', '2–4 hrs', '4–6 hrs', '6+ hrs']
    dff_h3 = dff.copy()
    dff_h3['social_media_category'] = pd.cut(
        dff_h3['social_media_hours'], 
        bins=bins_h3, 
        labels=labels_h3, 
        right=False,
        include_lowest=True
    )
    score_by_usage = dff_h3.groupby('social_media_category', observed=True)['exam_score'].mean().reset_index()
    if score_by_usage.empty:
        social_exam_fig = go.Figure()
        social_exam_fig.update_layout(title="H3: No data for social media analysis", height=500, paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
    else:
        social_exam_fig = px.pie(
            score_by_usage,
            values='exam_score',
            names='social_media_category',
            hole=0.5,
            title='Average Exam Score by Social Media Usage',
            color_discrete_sequence=['#00897B', '#F9A825', '#8E24AA', '#E64A19']
        )
        social_exam_fig.update_traces(textposition='inside', textinfo='label+percent+value', textfont=dict(color='white', size=14))
        social_exam_fig.update_layout(template='plotly_white', title_x=0.5, height=500, showlegend=True, paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')

    # H7
    dff_h7 = dff.groupby('age')['social_media_hours'].agg(mean='mean', std='std').reset_index()
    dff_h7['std_upper'] = dff_h7['mean'] + dff_h7['std']
    dff_h7['std_lower'] = (dff_h7['mean'] - dff_h7['std']).clip(lower=0) 
    age_social_fig = go.Figure()
    age_social_fig.add_trace(go.Scatter(
        x=dff_h7['age'].tolist() + dff_h7['age'].tolist()[::-1],
        y=dff_h7['std_upper'].tolist() + dff_h7['std_lower'].tolist()[::-1],
        fill='toself',
        fillcolor=f"rgba{px.colors.hex_to_rgb(COLORS['accent6']) + (0.2,)}",
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=False,
    ))
    age_social_fig.add_trace(go.Scatter(
        x=dff_h7['age'],
        y=dff_h7['mean'],
        mode='lines+markers',
        line=dict(color=COLORS['accent6'], width=3),
        marker=dict(size=8),
        name='Average Hours',
        hovertemplate="<b>Age: %{x}</b><br>Avg. Hours: %{y:.2f}<extra></extra>"
    ))
    age_social_fig.update_layout(
        title='H7: Average Social Media Hours by Age (with Std. Dev.)',
        xaxis_title='Age',
        yaxis_title='Avg. Social Media Hours',
        template="plotly_white", 
        height=500, 
        paper_bgcolor='rgba(0,0,0,0)', 
        plot_bgcolor='rgba(0,0,0,0)',
        xaxis=dict(type='category')
    )
    
    # H9
    try:
        bins_h9 = [0, 4, 7, 10]
        labels_h9 = ['Low', 'Medium', 'High']
        dff_h9 = dff.copy() 
        dff_h9['mental_health_binned'] = pd.cut(
            dff_h9['mental_health_rating'],
            bins=bins_h9,
            labels=labels_h9,
            right=True,
            include_lowest=True
        ).astype(str)
        treemap_data = dff_h9.groupby(['extracurricular_participation', 'mental_health_binned'], observed=True).agg(
            student_count=('extracurricular_participation', 'size'),
            avg_mental_health=('mental_health_rating', 'mean')
        ).reset_index()
        
        if treemap_data.empty:
            raise Exception("No data after grouping")

        extracurricular_fig = px.treemap(
            treemap_data,
            path=[px.Constant("All Students"), 'extracurricular_participation', 'mental_health_binned'],
            values='student_count',
            color='avg_mental_health',
            color_continuous_scale='RdBu',
            color_continuous_midpoint=5.5,
            range_color=[1, 10],
            title='H9: Student Mental Health by Extracurricular Participation'
        )
        extracurricular_fig.update_layout(
            template='plotly_white',
            title_x=0.5,
            margin=dict(t=50, l=25, r=25, b=25),
            paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)',
            height=500 
        )
        extracurricular_fig.update_traces(
            textinfo="label+value+percent root",
            marker=dict(line=dict(width=2, color='white')),
            insidetextfont=dict(size=14, color='#333333'),
            hovertemplate="<b>%{label}</b><br>Participates: %{parent}<br>Count: %{value}<br>Avg. MH Rating: %{color:.3f}<br>% of Total: %{percentRoot:.1%}<extra></extra>"
        )
    except Exception as e:
        extracB_fig = go.Figure()
        extracB_fig.update_layout(title="H9: Insufficient data to display treemap", height=500, paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
    
    return social_exam_fig, age_social_fig, extracurricular_fig

@app.callback(
    [Output("part-time-job-chart", "figure"),
     Output("internet-quality-chart", "figure")],
    [Input("gender-filter", "value"),
     Input("age-slider", "value"),
     Input("job-filter", "value")]
)
def update_socioeconomic_charts(genders, age_range, job_status):
    dff = get_filtered_data(genders, age_range, job_status)
    if dff.empty:
        return go.Figure(), go.Figure()

    # H8
    job_fig = px.histogram(
        dff, x="exam_score", color="part_time_job",
        marginal="box", nbins=20, barmode="overlay",
        title="Comparison of Exam Score Distributions"
    )
    job_fig.update_layout(
        xaxis_title="Exam Score", yaxis_title="Count of Students",
        template="plotly_white", height=350,
        paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)'
    )

    # H10
    category_order = {"internet_quality": ["Poor", "Average", "Good"]}
    internet_fig = px.violin(
        dff,
        x='internet_quality',
        y='exam_score',
        color='internet_quality',
        category_orders=category_order,
        box=True,
        points=False,
        title='Exam Score Distribution by Internet Quality'
    )
    internet_fig.update_layout(
        title_x=0.5,
        template='plotly_white',
        height=500,
        paper_bgcolor='rgba(0,0,0,0)', 
        plot_bgcolor='rgba(0,0,0,0)',
        showlegend=False
    )
    
    return job_fig, internet_fig

## SECTION 10: APP EXECUTION

Executes the Dash server. Note: jupyter_mode='tab' will open the dashboard in a new browser tab automatically when run.

In [15]:
if __name__ == "__main__":
    port = 8052
    # The 'jupyter_mode' parameter handles opening the tab automatically
    url = f"http://127.0.0.1:{port}/"
    webbrowser.open_new_tab(url)
    app.run(debug=True, port=port)